In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu

class Net(nn.Module):
    def __init__(self, kernel_size=5):
        super().__init__()
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, 6, kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        self.fc1 = nn.Linear(16 * conv_width * conv_height, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, print_results=False)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, optimizer=None, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net = net_generator()
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size=32)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

results = cross_validate_net(Net, dataset, repeat=3, epochs=10)
print_validation_results(results)

Training network 1 ...
Final loss: 1.6483, final accuracy: 72.26 %
	ALBATROSS: 60.71%
	BALD EAGLE: 78.79%
	BARN OWL: 65.38%
	EURASIAN MAGPIE: 76.32%
	FLAMINGO: 75.86%
	MALLARD DUCK: 70.37%
	OSTRICH: 86.36%
	PEACOCK: 80.00%
	PELICAN: 50.00%
	TRUMPTER SWAN: 64.29%
Training network 2 ...
Final loss: 1.6349, final accuracy: 65.41 %
	ALBATROSS: 33.33%
	BALD EAGLE: 62.50%
	BARN OWL: 86.36%
	EURASIAN MAGPIE: 60.71%
	FLAMINGO: 53.85%
	MALLARD DUCK: 58.82%
	OSTRICH: 92.00%
	PEACOCK: 86.67%
	PELICAN: 54.29%
	TRUMPTER SWAN: 71.43%
Training network 3 ...
Final loss: 1.4828, final accuracy: 70.21 %
	ALBATROSS: 46.88%
	BALD EAGLE: 74.29%
	BARN OWL: 78.57%
	EURASIAN MAGPIE: 72.00%
	FLAMINGO: 80.00%
	MALLARD DUCK: 67.65%
	OSTRICH: 75.00%
	PEACOCK: 90.62%
	PELICAN: 43.48%
	TRUMPTER SWAN: 70.59%
Losses:  [1.6483167935392424, 1.634927494987233, 1.4828458224257377]
Loss:  mean: 1.5887, std: 0.0919
Accuracies:  [0.7226027397260274, 0.6541095890410958, 0.702054794520548]
Accuracy:  mean: 0.6929, std: 0.0351

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/test1.csv")